In [15]:
import numpy as np 
import pandas as pd
import statsmodels.formula.api as smf
import statsmodels.api as sm

from numpy.random import default_rng
rng = default_rng()

N=50000

df = pd.DataFrame({
    'const': 1,
    'a':rng.normal(-1,1, size=N),
    'b':rng.normal(-1,1,size=N),
    'c':rng.normal(-1,1,size=N),
    'i':0,  
})

df['i'] =  df.a * df.b

beta = [1, 1 ,1 ,1 ,.5]

df['y']= np.dot(df[['const','a','b','c','i']], beta)

err_std = 1 # Error std as a portion of the signal std
df['y'] = df.y + err_std* df.y.std() * rng.normal(size=N)

r1 = smf.ols('y ~ a + b + c', data=df).fit()
r2 = smf.ols('y ~ a + b + c + a:b', data=df).fit()

print(r1.rsquared, r2.rsquared)
pd.DataFrame([r1.params, r2.params]).append(pd.Series(beta, index=r2.params.index), ignore_index=True)

0.428577450407555 0.4992538479697416


,Intercept,a,b,c,a:b
0,0.489955,0.493514,0.495887,1.003022,NaN
1,0.988408,0.988812,0.994253,1.004311,0.499366
2,1.000000,1.000000,1.000000,1.000000,0.500000


In [5]:
r2.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                      y   R-squared:                       0.941
Model:                            OLS   Adj. R-squared:                  0.941
Method:                 Least Squares   F-statistic:                 1.995e+05
Date:                Sat, 27 Nov 2021   Prob (F-statistic):               0.00
Time:                        07:23:58   Log-Likelihood:                -18979.
No. Observations:               50000   AIC:                         3.797e+04
Df Residuals:                   49995   BIC:                         3.801e+04
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      1.0017      0.004    281.580      0.000       0.995       1.009
a              0.9987      0.002    445.894      0.000       0.994       1.003
b              0.9994      0.002    440.177      0.000       0.995       1.004
c              0.9996      0.002    634.146      0.000       0.997       1.003
a:b            0.9978      0.002    628.197      0.000       0.995       1.001
==============================================================================
Omnibus:                        2.011   Durbin-Watson:                   2.007
Prob(Omnibus):                  0.366   Jarque-Bera (JB):                1.996
Skew:                           0.014   Prob(JB):                        0.369
Kurtosis:                       3.012   Cond. No.                         7.58
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [36]:
import random

df = pd.DataFrame({
    'const': 1,
    'sex':random.choices(list('mf'), k=N),
    'race':random.choices(list('wb'), k=N),     
})

df['income'] = 75_000
df.loc[df.sex=='m','income'] += 10_000
df.loc[df.race=='w','income'] += 10_000
df.loc[(df.race=='w')&(df.sex=='m'),'income'] += 10_000

err_std = 1 # Error std as a portion of the signal std
df['income'] = df.income + err_std* df.income.std() * rng.normal(size=N)

r1 = smf.ols('income ~ C(sex) + C(race)', data=df).fit()
r2 = smf.ols('income ~ C(sex) + C(race) + C(sex):C(race)', data=df).fit()

print(r1.rsquared, r2.rsquared)
pd.DataFrame([r1.params, r2.params])

0.47170844636334475 0.4999419036822619


,Intercept,C(sex)[T.m],C(race)[T.w],C(sex)[T.m]:C(race)[T.w]
0,72565.729919,14869.437199,15099.549987,NaN
1,75144.575889,9718.107714,9946.581586,10343.991725
